<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado desde 2017-2 al 2024-2 por Equipo Docente IIC2233</font>
</p>

# Tabla de contenidos

1. [Introducción](#Introducción)
2. [Comando type](#Comando-type)
3. [Metaclases personalizadas](#Metaclases-personalizadas)
    1. [Asignación](#Asignación-de-metaclases)
    2. [Creación](#Creación-de-metaclases)
        1. [new](#Método-\_\_new__)
        2. [init](#Método-\_\_init__)
    3. [Creación de instancias](#Creación-de-instancias)
4. [Ejemplos](#Ejemplos)

# Introducción

Como vimos en los contenidos de introducción a la programación, en Python podemos guardar diferentes tipos de datos dentro de variables. Por ejemplo, podemos guardar enteros, *strings* y listas:

In [1]:
x = 10
y = 20
z = x + y  # 30

string_ejemplo = 'Bienvenidos a IIC2233'

lista_ejemplo = [10, 'variable', True, 5.5]

Adicionalmente, podemos usar variables para guardar resultados de funciones y métodos o instancias de clases.

In [2]:
def suma(a: int, b: int):
    return a + b

total = suma(x, y)

In [7]:
class Persona:
    def __init__(self, nombre: str, apellido: str):
        self.nombre = nombre
        self.apellido = apellido
        self.edad = 60

    def __str__(self):
        return f'Mi nombre es {self.nombre} {self.apellido}'

    def entregar_edad(self):
        return self.edad


maradona = Persona("Diego", "Maradona")
edad = maradona.entregar_edad()

print(maradona)
print(edad)

Mi nombre es Diego Maradona
60


Un aspecto que hasta ahora no hemos visto, es que también podemos guardar clases en variables.

In [9]:
class Perro: 
    def __init__(self, nombre: str):
        self.nombre = nombre

clase_perro = Perro

instancia_perro = clase_perro('Coraje')
print(instancia_perro.nombre)

Coraje


Siguiendo la misma línea, todos los elementos que se pueden guardar en variables, pueden ser entregados como argumento a una función. Esto incluye los tipos que utilizamos tradicionalmente, pero según lo que acabamos de ver, deberíamos poder entregar no sólo instancias de clases, si no que las clases mismas.

In [10]:
from typing import Any

def imprimir_dato(dato: Any):
    print(dato)

In [11]:
# Enteros y listas
print('Enteros y Listas:')
imprimir_dato(112)
imprimir_dato(lista_ejemplo)

# Instancias
print('\nInstancias:')
imprimir_dato(maradona)
imprimir_dato(instancia_perro)

# Clases
print('\nClases:')
imprimir_dato(Persona)
imprimir_dato(Perro)
imprimir_dato(clase_perro)

Enteros y Listas:
112
[10, 'variable', True, 5.5]

Instancias:
Mi nombre es Diego Maradona

Clases:
<class '__main__.Persona'>
<class '__main__.Perro'>
<class '__main__.Perro'>


La razón por la que todos estos elementos se pueden guardar en variables y entregar como argumentos es porque tienen un tipo de dato, es decir, son instancias de una clase.

Por ejemplo, los enteros son instancias de la clase `int`, las listas son instancias de la clase `list`, Maradona es una instancia de la clase `Persona` y Coraje es una instancia de la clase `Perro`.

Entonces, ¿de qué objeto son instancias las clases `Persona` y `Perro`?.
A las clases que definen el tipo de otras clases, en Python, las llamamos **Metaclases**. En otras palabras, una metaclase es una *clase* cuyas instancias son otras *clases*.

# Comando type

El comando `type()` recibe una argumento y retorna su tipo de dato, es decir, la clase a la que pertenece la instancia.

In [12]:
from typing import Any

def imprimir_tipo_de_dato(dato: Any):
    print(type(dato))

In [13]:
# Enteros y listas
print('Enteros y Listas:')
imprimir_tipo_de_dato(112)
imprimir_tipo_de_dato(lista_ejemplo)

# Instancias
print('\nInstancias:')
imprimir_tipo_de_dato(maradona)
imprimir_tipo_de_dato(instancia_perro)

Enteros y Listas:
<class 'int'>
<class 'list'>

Instancias:
<class '__main__.Persona'>
<class '__main__.Perro'>


Luego, si el objeto que entregamos es una clase, `type()` retornará cuál es la metaclase a la que pertenece esa clase.

In [8]:
# Clases
print('\nClases:')
imprimir_tipo_de_dato(Persona)
imprimir_tipo_de_dato(Perro)
imprimir_tipo_de_dato(clase_perro)


Clases:
<class 'type'>
<class 'type'>
<class 'type'>


En este ejemplo, podemos ver que el tipo de dato, tanto para `Perro` como para `Persona`, es `type`. En otras palabras `type` es la Metaclase de estas dos clases. 

Lo anterior se debe a que la metaclase por defecto en Python es `type` y, dado que en ningún momento especificamos la metaclase que queríamos usar, esta fue asignada.

# Metaclases personalizadas

### Asignación de metaclases
Para declarar que una clase tiene un cierto tipo, debemos agregar el _keyworded argument_ `metaclass={tipo_de_clases}`. 


In [21]:
class Gato(metaclass=type):
    def __init__(self, nombre):
        self.nombre = nombre
        
instancia_gato = Gato('Cachirulo')
instancia_gato

### Creación de metaclases
Ahora que sabemos asignar metaclases, solo necesitamos poder crear nuestras propias metaclases para poder utilizarlas.

En Python, las metaclases se declaran como clases que heredan de `type` y su principal propósito es **controlar la creación e instanciación de una clase**. Para realizar esto, necesitamos implementar, en la metaclase, los métodos `__new__` e `__init__`, respectivamente. 

#### Método \_\_new__
El método `__new__` debe ser implementado cuando queremos controlar la creación de una nueva clase. Este método tendrá los siguientes parámetros:

* `cls`: De manera similar a como `self` representa la instancia a actual, este argumento **representa la Meta Clase actual**.
* nombre: Corresponde a un `str` con el nombre de la clase.
* clases_base: Es una tupla que contiene las clases de las que hereda la clase instanciada.
* diccionario: Como dice su nombre, es un diccionario que contiene la información de la clase instanciada.

Utilizando este método podemos, por ejemplo, imprimir el nombre de la clase creada.

In [22]:
class MetaClaseImpresora(type):
    def __new__(cls, nombre, clases_base, diccionario):
        # Imprimimos los mensajes que queremos
        print(f'Llamando al método __new__ de {cls.__name__}')
        print(f'Creando la clase {nombre}')
        
        # Creamos la clase usando el __new__ de type
        clase = super().__new__(cls, nombre, clases_base, diccionario)
        
        # Retornamos la clase creada, sin ninguna modificación
        return clase

Es importante señalar que el `__new__` de la Metaclase **se llamará al momento de crear la clase**, **NO** al momento de crear instancias de la clase. Esto lo podemos ver claramente en el siguiente ejemplo: 

In [23]:
print('Aquí creamos la clase, por lo que se llama a new\n')

class Gato(metaclass=MetaClaseImpresora):
    def __init__(self, nombre):
        self.nombre = nombre
    
    def __call__(self):
        print(self.nombre)

Aquí creamos la clase, por lo que se llama a new

Llamando al método __new__ de MetaClaseImpresora
Creando la clase Gato


In [24]:
print('Aquí instanciamos la clase, por lo que NO se llama a new')

instancia_gato = Gato('Luna')

Aquí instanciamos la clase, por lo que NO se llama a new


Utilizando el método `__new__`, podemos además agregar comportamientos mucho más interesantes y complejos. 

A continuación, nuestra metaclase cambia el nombre de la clase creada, agrega un atributo nuevo, y fuerza una herencia sobre la clase.

In [25]:
class Guerrero:
    def __init__(self, nombre):
        self.nombre = nombre

class MetaTransformaPaladin(type):
    def __new__(cls, nombre, clases_base, diccionario):
        # Imprimimos un mensaje
        print(f'Llamando al método __new__ de {cls.__name__}')
        
        # Cambiamos el nombre de la clase
        print(f'Intentaron crear la clase {nombre}, pero en su lugar crearemos Paladin')
        nuevo_nombre = 'Paladin'
             
        # Cambiamos las clases base
        print(f'Intentaron heredar de {clases_base}, pero Paladin hereda de Guerrero')
        nuevas_clases_base = (Guerrero,)
        
        # Actualizamos el diccionario con el nuevo atributo
        print('Los paladines deben tener el atributo "defensa" como atributo de clase')
        diccionario['defensa'] = 100
        
        # Llamamos al __new__ de type con los nuevos argumentos
        clase = super().__new__(cls, nuevo_nombre, nuevas_clases_base, diccionario)
        print(clase)
        return clase

In [26]:
class Mago(metaclass=MetaTransformaPaladin):
    def __init__(self, nombre):
        self.nombre = nombre

Llamando al método __new__ de MetaTransformaPaladin
Intentaron crear la clase Mago, pero en su lugar crearemos Paladin
Intentaron heredar de (), pero Paladin hereda de Guerrero
Los paladines deben tener el atributo "defensa" como atributo de clase
<class '__main__.Mago'>


In [27]:
instancia_mago = Mago('Morgana')

print(instancia_mago.__class__.__name__)
print(isinstance(instancia_mago, Guerrero))
print(instancia_mago.defensa)

Paladin
True
100


#### Método \_\_init__
El método `__init__` debe ser implementado cuando queremos controlar **la instanciación** de una nueva clase.

Tiene casi los mismos parámetros que `__new__`, pero en lugar de recibir el `cls` que representa la metaclase, volveremos al `self`, que representará la instancia de la metaclase, es decir, la clase actual. 

La principal diferencia es que este método es llamado **después de que la clase haya sido creada**, por esto, si bien podemos agregar funciones y atributos a la clase dentro del `__init__` de la metaclase, no podremos modificar cosas propias de la clase como su nombre o sus clases base.

Utilizando este método podemos recrear la metaclase impresora, pero no podremos hacer lo mismo que la metaclase usada para el Paladín.

In [28]:
class MetaClaseImpresoraInit(type):
    def __init__(self, nombre, clases_base, diccionario):
        # Imprimimos los mensajes que queremos
        print(f'Llamando al método __init__ de MetaClaseImpresoraInit')
        print(f'Creando la clase {nombre}')
        
        # Creamos la clase usando el __init__ de type
        clase = super().__init__(nombre, clases_base, diccionario)
        
        # Retornamos la clase creada, sin ninguna modificación
        return clase

In [29]:
class Gato(metaclass=MetaClaseImpresoraInit):
    def __init__(self, nombre):
        self.nombre = nombre
    
    def __call__(self):
        print(self.nombre)
        
print('\nCreando instancia de gato:')
instancia_gato = Gato('Grumo')
instancia_gato()

Llamando al método __init__ de MetaClaseImpresoraInit
Creando la clase Gato

Creando instancia de gato:
Grumo


In [30]:
class MetaAgregaDefensa(type):
    def __init__(self, nombre, clases_base, diccionario):
        # Imprimimos un mensaje
        print(f'Llamando al método __init__ de MetaAgregaDefensa')
               
        # Actualizamos el diccionario agregando el atributo defensa
        print('Los gatos también merecen defenderse')
        self.defensa = 100

        clase = super().__init__(nombre, clases_base, diccionario)
        return clase

In [31]:
class GatoDefensivo(metaclass=MetaAgregaDefensa):
    def __init__(self, nombre):
        self.nombre = nombre
    
    def __call__(self):
        print(f'Gato de nombre {self.nombre} con atributo de clase defensa = {self.defensa}')
        
print('\nCreando instancia de gato:')
instancia_gato = GatoDefensivo('Pepe')
instancia_gato()

Llamando al método __init__ de MetaAgregaDefensa
Los gatos también merecen defenderse

Creando instancia de gato:
Gato de nombre Pepe con atributo de clase defensa = 100


### Creación de instancias

Como podrán haber notado en algunos de los ejemplos anteriores, estábamos tratando las instancias de la clase gato como funciones, haciendo `instancia_gato()`. Al realizar esto, se llamará al método `__call__` definido en la clase y llamamos *callable* a los objetos cuyo método `__call__` se encuentra implementado.

Si aplicamos esta misma idea a las clases, cuando queremos crear una nueva instancia de una clase, comúnmente lo hacemos a través de `NombreClase()`. Si bien esta es la manera esperada, ¿por qué no utilizamos `NombreClase.__init__()`?. La razón es que al utilizar directamente el nombre de la clase, lo que estamos haciendo es llamar al `__call__` de la metaclase, que luego se encargará de llamar al `__init__` de la clase y así crear la instancia correctamente.

Luego, de lo anterior podemos concluir que si queremos afectar la creación de instancias de una clase, podemos hacerlo modificando el `__call__` de su metaclase. Este método recibe como argumentos el `cls` y además todos los argumentos que se entreguen a la llamada; dado que esto puede ser una cantidad arbitraria que dependerá de cada clase, podemos acomodarnos a todas utilizando `*args` y `**kwargs`.

A continuación tenemos una metaclase que imprimirá un mensaje cada vez que se cree una instancia de sus clases correspondientes.

In [32]:
class MetaImprimeInstancias(type):
    def __call__(cls, *args, **kwargs):
        print('LLamando a __call__ de MetaImprimeInstancias')
        print(f'Instanciando la clase {cls.__name__} con args: {args} y kwargs: {kwargs}')
        instancia = super().__call__(*args, **kwargs)
        return instancia

In [33]:
class GatoImpresor(metaclass=MetaImprimeInstancias):
    def __init__(self, nombre, edad, personalidad):
        self.nombre = nombre
        self.edad = edad
        self.personalidad = personalidad
    
    def __call__(self):
        print(f'Gato de nombre {self.nombre} con personalidad {self.personalidad}')


instancia_gato = GatoImpresor('Mia', 6, 'Amigable')
instancia_gato()

LLamando a __call__ de MetaImprimeInstancias
Instanciando la clase GatoImpresor con args: ('Mia', 6, 'Amigable') y kwargs: {}
Gato de nombre Mia con personalidad Amigable


# Ejemplos

A continuación se muestran un par de ejemplos más realistas para el uso de metaclases.

### Clase que no puede ser subclaseada

Definamos una metaclase que asegure que ninguna clase va a poder heredar de la metaclase. Se podrán crear clases que tengan la metaclase `MetaFinal` pero no será posible que estas clases sean la clase base de otras.

In [34]:
class MetaFinal(type):
    def __new__(cls, nombre, clases_base, diccionario):
        # Revisamos las clases base de la clase que se está creando
        for clase in clases_base:
            # Si la clase es instancia de la metaclse, no puede ser creada
            if isinstance(clase, cls):
                print(f'No puede crearse la clase {nombre}')
                return None
        print(f'Clase {nombre} creada correctamente')
        return super().__new__(cls, nombre, clases_base, diccionario)

In [35]:
class NoHeredable(metaclass=MetaFinal):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
instancia_no_heredable = NoHeredable()
print(instancia_no_heredable)

Clase NoHeredable creada correctamente


In [36]:
class ClaseImposible(NoHeredable):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

instancia_imposible = ClaseImposible()

No puede crearse la clase ClaseImposible


TypeError: 'NoneType' object is not callable

### Singletón

En este ejemplo veremos cómo definir una metaclase que asegure que cada vez que se llama a la clase para crear una nueva instancia, ésta retorne la nueva instancia, si y sólo si, ninguna instancia de la clase se ha creado antes. En otro caso, se debería retornar la misma instancia creada la primera vez:

In [37]:
class MetaSoloUna(type):
    def __call__(cls, *args, **kwargs):
        if not hasattr(cls, 'instance'):
             cls.instance = super().__call__(*args, **kwargs)
        return cls.instance

In [38]:
class SoloUna(metaclass=MetaSoloUna):
    pass


instancia_1 = SoloUna()
print("instancia_1.instance == None: ", instancia_1.instance == None)

instancia_2 = SoloUna()
print("instancia_1 is instancia_2 ", instancia_1 is instancia_2)
print("id instancia_1: ", id(instancia_1))
print("id instancia_2: ", id(instancia_2))


instancia_1.instance == None:  False
instancia_1 is instancia_2  True
id instancia_1:  139625231640720
id instancia_2:  139625231640720
